# Calculating percentage of total in Spark

See Stackoverflow [here](https://stackoverflow.com/questions/43473334/how-to-calculate-percentage-over-a-dataframe)

In [1]:
from pyspark.context import SparkContext, SparkConf
from pyspark.sql import SparkSession, Window
import pyspark.sql.functions as F

conf=SparkConf()


sc = SparkContext.getOrCreate(conf=conf)
spark = SparkSession(sc)


In [2]:
## For each group, we want to get percentage of total

In [3]:
import numpy as np
import pandas as pd

num_rows = 1000
category = np.random.choice(["a", "b", "c"],num_rows, p=[0.1, 0.3, 0.6])
values = np.random.randint(1, 100, num_rows)
pd_df = pd.DataFrame({"cat": category, "val": values})


In [4]:
df = spark.createDataFrame(pd_df)

In [5]:
df.registerTempTable("df")

sql = """
select cat, sum(val)/(select sum(val) from df) as perc
from df 
group by cat
"""

spark.sql(sql).write.csv("delete/", mode="overwrite")
# spark.sql(sql).show()

In [5]:
## Can we re-write this in pyspark

val_sum = df.groupBy().agg(F.sum("val"))

df_perc = df.select((df["val"]/val_sum.collect()[0][0]).alias("perc"))
df_perc.write.csv("delete3/", mode="overwrite")


In [7]:
df.registerTempTable("df")

sql = """
select val/sum(val) over () as perc
from df

"""

spark.sql(sql).write.csv("delete2/", mode="overwrite")